In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2 as cv
# import seaborn as sns
import pandas as pd
from skimage.filters import sobel
from skimage.feature import greycomatrix, greycoprops
from skimage.measure import shannon_entropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
from PIL import Image
from tqdm.notebook import tqdm

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

In [ ]:
df = pd.read_csv("features.csv")
dt = pd.read_csv("Data_test.csv")
df.head()

In [ ]:
df.drop(['filenames'], axis=1, inplace=True)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.head()

In [ ]:
label_distr = df['output'].value_counts()

label_name = ['Normal','RS','MAS','WS']


print(label_distr)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

In [ ]:
def decimal_scaling(data):
    data = np.array(data, dtype=np.float32)
    max_row = data.max(axis=0)
    c = np.array([len(str(int(number))) for number in np.abs(max_row)])
    return data/(10**c)

X = decimal_scaling(df[['Energy_0', 'Corr_0', 'Homogen_0', 'Contrast_0', 'ASM_0',
                        'Energy_45', 'Corr_45', 'Homogen_45', 'Contrast_45', 'ASM_45',
                        'Energy_90', 'Corr_90', 'Homogen_90', 'Contrast_90', 'ASM_90',
                        'Energy_135', 'Corr_135', 'Homogen_135', 'Contrast_135', 'ASM_135']].values)

In [ ]:
le = LabelEncoder()
le.fit(df["output"].values)

print(" categorical label : \n", le.classes_)

Y = le.transform(df['output'].values)
Y = to_categorical(Y)

print("\n\n one hot encoding for sample 0 : \n", Y[0])

In [ ]:
X_train, X_test, y_train, y_test = \
                    train_test_split(X, 
                                     Y, 
                                     test_size=0.2, 
                                     random_state=42)
  
print("Dimensi data :\n")
print("X train \t X test \t Y train \t Y test")  
print("%s \t %s \t %s \t %s" % (X_train.shape, X_test.shape, y_train.shape, y_test.shape))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

import keras
from keras import backend as K

In [ ]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def nn_model(max_len):
    
    model = Sequential()
    model.add(Dense(32,
                    activation="elu",
                    input_shape=(max_len,)))
    model.add(Dense(128, activation="elu"))
    model.add(Dense(64, activation="elu"))
    model.add(Dense(32, activation="elu"))
    model.add(Dense(16, activation="elu"))
    model.add(Dense(4))
    model.add(Activation("sigmoid"))
    
    model.summary()
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics = ['accuracy', precision, recall])
    return model

def check_model(model_, x, y, x_val, y_val, epochs_, batch_size_):
    
    hist = model_.fit(x,
                      y,
                      epochs=epochs_,
                      batch_size=batch_size_,
                      validation_data=(x_val,y_val))
    return hist

In [ ]:
max_len = X_train.shape[1]

EPOCHS = 500
BATCH_SIZE = 32

model = nn_model(max_len)
history=check_model(model, X_train,y_train,X_test,y_test, EPOCHS, BATCH_SIZE)

In [ ]:
hist_df = pd.DataFrame(history.history)
hist_df['epoch'] = hist_df.index + 1
cols = list(hist_df.columns)
cols = [cols[-1]] + cols[:-1]
hist_df = hist_df[cols]
hist_df.head(500)

In [ ]:
values_of_best_model = hist_df[hist_df.val_loss == hist_df.val_loss.min()]
values_of_best_model

In [ ]:
model.summary()

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(10, 10))
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
 # predict test data
y_pred=model.predict_proba(X_test)
print(y_pred)

In [ ]:
predict = []
for i in range(0,16):
    predict.append(max(y_pred[i]))
predict

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
np.set_printoptions(precision=2)

In [ ]:
# Plot non-normalized confusion matrix
plot_confusion_matrix(cnf_matrix,
                      classes=['Normal','RS','MAS','WS'],
                      normalize=False,
                      title='Confusion matrix, with normalization')

In [ ]:
print(classification_report(y_test.argmax(axis=1),
                            y_pred.argmax(axis=1),
                            target_names=['Normal','RS','MAS','WS']))

In [ ]:
ytest = dt['output']
xtest = dt.drop(columns=['Unnamed: 0', 'output','filenames'])
xtest = preprocessing.StandardScaler().fit(xtest).transform(xtest.astype(float))
dt.head()

In [ ]:
le = LabelEncoder()
le.fit(dt["output"].values)

print(" categorical label : \n", le.classes_)

y2 = le.transform(dt['output'].values)
y2 = to_categorical(y2)

print("\n\n one hot encoding for sample 0 : \n", y2[0])

In [ ]:
y2test = y2

In [ ]:
xtest.shape, y2test.shape

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
ypredict = model.predict(xtest)
print (ypredict)

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y2test.argmax(axis=1), ypredict.argmax(axis=1))
np.set_printoptions(precision=2)

In [ ]:
# Plot non-normalized confusion matrix
plot_confusion_matrix(cnf_matrix,
                      classes=['Normal','RS','MAS','WS'],
                      normalize=False,
                      title='Confusion matrix, with normalization')

In [ ]:
print(classification_report(y2test.argmax(axis=1),
                            ypredict.argmax(axis=1),
                            target_names=['Normal','RS','MAS','WS']))

In [ ]:
import gradio as gr
import cv2

from sklearn.cluster import KMeans
from skimage.feature import greycomatrix, greycoprops

In [ ]:
def preprocessing(img):
    resized = cv2.resize(img, (256,256),Image.BICUBIC)
    bgr = resized[:, :, ::-1].copy()

    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    clahe = cv2.createCLAHE(clipLimit=0)
    lab[:,:,0] = clahe.apply(lab[:,:,0])
    lab[:,:,1] = clahe.apply(lab[:,:,1])
    lab[:,:,2] = clahe.apply(lab[:,:,2])
    bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    res = cv2.fastNlMeansDenoisingColored(bgr, None, 10, 10, 1, 3)
        #bgr = cv2.bilateralFilter(bgr, 5, 1, 1)
    return res

def segmentation(img, online=False):
    pre = preprocessing(img)
    image = cv2.cvtColor(pre, cv2.COLOR_BGR2LAB)
    image = image.reshape((256*256,3))

    clt = KMeans(n_clusters = 5)
    labels = clt.fit_predict(image)
    quant = clt.cluster_centers_.astype("uint8")[labels]
    quant = quant.reshape((256, 256, 3))
    res = image.reshape((256, 256, 3))

    if online:
        return quant

    return res

def gray_segmentation(img):
    pre = preprocessing(img)
    segmented = segmentation(pre, True)
    bgr = cv2.cvtColor(segmented, cv2.COLOR_LAB2BGR)
    res = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    return res

def extract(img, online=False):
    image= gray_segmentation(img)
    degs = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    post = [0, 45, 90, 135]
    val = []
    for i,deg in enumerate(degs):
        GLCM = greycomatrix(image, [3], [deg])
        energy = greycoprops(GLCM, 'energy')[0]
        corr = greycoprops(GLCM, 'correlation')[0]
        hom = greycoprops(GLCM, 'homogeneity')[0]
        contr = greycoprops(GLCM, 'contrast')[0]
        asm = greycoprops(GLCM, 'ASM')[0]
        
        temp = [post[i], energy[0], corr[0], hom[0], contr[0], asm[0]]
        val.append(temp)

    res = pd.DataFrame(
        np.array(val),
        columns= ['Degree', 'Contrast', 'Energy', 'Korelasi', 'ASM', 'Homogenity'],
    )

    if online:
        return np.array(val)
    return res

def predict_(img):
    feature = extract(img, True)[:, 1:]
    # print(feature.shape)
    target_names=['Normal','RS','MAS','WS']
    feature = feature.reshape((1,20))
    pred = model.predict(feature)

    return target_names[pred.argmax()]



In [ ]:
demo = gr.Blocks()

with demo:
    gr.Markdown('Step by step Demo')
    img_input = gr.Image()
    # with gr.Row():
    img_preprocess = gr.Image()
    button_preprocess = gr.Button('Preprocessing Button')

    # with gr.Row():
    img_segmentation = gr.Image()
    button_segm = gr.Button("Segmentation Button")
    
    # with gr.Row():
    img_grayscale = gr.Image()
    button_gray = gr.Button("RGB to Grayscale Button")

    # with gr.Row():
    data = gr.DataFrame(
        headers= ['Degree','Contrast', 'Energy', 'Korelasi', 'ASM', 'Homogenity'],
            # row_count=(1, 'fixed')
        )
    button_extract = gr.Button('Extract Data')
    
    output = gr.Textbox(label="Prediction Result")
    button_predict = gr.Button('Predict')
    button_preprocess.click(preprocessing, inputs=img_input, outputs=img_preprocess)
    button_segm.click(segmentation, inputs=img_input, outputs=img_segmentation)
    button_gray.click(gray_segmentation, inputs=img_input, outputs=img_grayscale)
    button_extract.click(extract, inputs=img_input, outputs=data)
    button_predict.click(predict_, inputs=img_input, outputs=output)

In [ ]:
demo.close()
demo.launch()